# Redes Neuronales

In [500]:
from IPython.display import Image
import polars as pl
import os
import sys
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
device = 'cpu'
sys.path.append('/home/sebacastillo/neuralnets/')
from src.utils import get_project_root
root = get_project_root()

In [599]:
OR_problem = False
if OR_problem:
   df = np.loadtxt(str(root) + '/data/OR.csv',
                 delimiter=",")
else:
   df = np.loadtxt(str(root) + '/data/XOR.csv',
                 delimiter=",")

# Percentron original (Rosenblatt: 1962)

Bibliografía y fuentes:   
- [Introduction to Machine Learning](https://openlearninglibrary.mit.edu/courses/course-v1:MITx+6.036+1T2019/course/)

In [699]:
np.random.shuffle(df)

def ttsplit(data, prop=0.2):
    assert isinstance(data, np.ndarray), "Input should be a NumPy array"
    p = int(len(data) * prop)
    test = data[:p, :]
    train = data[p:, :]
    X_train = train[:,:-1]
    y_train = train[:,-1]
    X_test = test[:,:-1]
    y_test = test[:,-1]
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = ttsplit(df, 0.2)

In [700]:
# Data imputs
input_data = X_train
true_labels = y_train

# Initialize the weights and bias
n = input_data.shape[0]
weights = np.ones(input_data.shape[1])*0.5
bias = 0

# Number of epochs (tau)
epochs = 100

# Training loop
for epoch in range(epochs):

    for i in range(n):
        
        x = input_data[i]
        y_true = true_labels[i]

        # Calculate the output using weights and bias. These is our hipothesis.        
        # the sign mater for the next conditional product.
        # The first output is always wrong! because all weight are set to 0
        output = np.dot(weights, x) + bias

        # Sign Function part where we compare the sign of the true label and the sign of our prediction.
        # If both y_true and output have the same sign the result is positive and the condition fail.
        # If it never fail it means the we hold the hipotesis (weight and bias).
        # If both have diferent sign the result of the product is negative and the codition is true.
        # So we should update the weights.
        if y_true * output <= 0:
            # Here is where learning happens as weight update
            weights += y_true * x
            bias += y_true

# The final weights (theta) and bias (theta_0)
print("Weights (theta):", weights)
print("Bias (theta_0):", bias)


Weights (theta): [ 0.01403 -0.30108]
Bias (theta_0): 0.0


In [701]:
# Exp.1:Acuracy: .76
#Weights (theta): [1.98073 1.32935]
#Bias (theta_0): 1.0
# Exp.2: Acuracy: .73
#Weights (theta): [-0.17513 -1.07761]
#Bias (theta_0): 1.0
# Exp.3: Acuracy: .71
#Weights (theta): [1.39779 1.20869]
#Bias (theta_0): 1.0
# Exp.4 con 200 epoch: Acuracy: 51: Bajo!
#Weights (theta): [-0.37849 -0.58772]
#Bias (theta_0): 0.0
# muchos con bajo accuracy
# Exp.muchos+1: Accuracy .74
#Weights (theta): [ 1.5557  -1.07645]
#Bias (theta_0): -1.0


def validate_perceptron(X_test, y_test, weights, bias):
    total_error = 0
    correct_predictions = 0
    n = X_test.shape[0]

    for i in range(n):
        x = X_test[i]
        y_true = y_test[i]

        # Calculate the output using weights and bias
        output = np.dot(weights, x) + bias
        y_pred = np.sign(output) if output != 0 else 1

        # Calculate the L2 loss
        error = (y_true - y_pred) ** 2
        total_error += error

        # Update the accuracy counter
        if y_pred == y_true:
            correct_predictions += 1

    # Calculate average error and accuracy
    average_error = total_error / n
    accuracy = correct_predictions / n

    return average_error, accuracy

In [702]:
# Validate the perceptron model
average_error, accuracy = validate_perceptron(X_test, y_test, weights, bias)

print("Average L2 Loss on Test Data:", average_error)
print("Accuracy on Test Data:", accuracy)

Average L2 Loss on Test Data: 2.0
Accuracy on Test Data: 0.5


# Perceptron con Descenso Gradiente

In [686]:
import numpy as np

def compute_gradient(y_true, y_pred, x):
    gradient_w = -2 * (y_true - y_pred) * x
    gradient_b = -2 * (y_true - y_pred)
    return gradient_w, gradient_b

def train_gradient_descent(input_data, true_labels, epochs, learning_rate):
    n = input_data.shape[0]
    weights = np.zeros(input_data.shape[1])
    bias = 0

    for epoch in range(epochs):
        for i in range(n):
            x = input_data[i]
            y_true = true_labels[i]

            # Calculate the output using weights and bias
            output = np.dot(weights, x) + bias
            y_pred = np.sign(output) if output != 0 else -1

            # Compute the gradient of the loss
            gradient_w, gradient_b = compute_gradient(y_true, y_pred, x)

            # Update the weights and bias
            weights -= learning_rate * gradient_w
            bias -= learning_rate * gradient_b

    return weights, bias

# Train the gradient descent-based perceptron model
epochs = 500
learning_rate = 0.1
weights, bias = train_gradient_descent(X_train, y_train, epochs, learning_rate)

print("Weights:", weights)
print("Bias:", bias)


Weights: [0.0252268 0.0183228]
Bias: 0.0


In [687]:
def validate_perceptron(X_test, y_test, weights, bias):
    total_error = 0
    correct_predictions = 0
    n = X_test.shape[0]

    for i in range(n):
        x = X_test[i]
        y_true = y_test[i]

        # Calculate the output using weights and bias
        output = np.dot(weights, x) + bias
        y_pred = np.sign(output) if output != 0 else 1

        # Calculate the L2 loss
        error = (y_true - y_pred) ** 2
        total_error += error

        # Update the accuracy counter
        if y_pred == y_true:
            correct_predictions += 1

    # Calculate average error and accuracy
    average_error = total_error / n
    accuracy = correct_predictions / n

    return average_error, accuracy

# Validate the perceptron model
average_error, accuracy = validate_perceptron(X_test, y_test, weights, bias)

print("Average L2 Loss on Test Data:", average_error)
print("Accuracy on Test Data:", accuracy)

Average L2 Loss on Test Data: 1.990909090909091
Accuracy on Test Data: 0.5022727272727273


# Teorema de convergencia del perceptron   

Convergence theorem
The basic result about the perceptron is that, if the training data  Dn  is linearly separable, then the perceptron algorithm is guaranteed to find a linear separator.



# Preguntas:    

- porqué inicializamos weights en 0.5?

# Ejemplo funcional

In [ ]:
def inicializar(num_features):
    #np.random.randn(num_features)
    w = np.concatenate(([0.0], np.ones(num_features) * 0.5))    
    return w

In [ ]:
w = inicializar(2)
w

array([0. , 0.5, 0.5])

In [ ]:
# Funciones de Activación
def sigmoid(z): 
    """
    Sigmoid activation function
    """ 
    return 1 / (1 + np.exp(-z))    

def sign(x, weights, bias):
    """
    Sign activation function    
    """ 
    # Calculate the weighted sum of the input features and bias
    weighted_sum = np.dot(weights, x) + bias
    # Apply the sign function to the weighted sum
    if weighted_sum >= 0:
        return 1
    else:
        return -1    

In [ ]:
def forward_pass(x, weights, bias, y_true):
    y_pred = sign(x, weights, bias)
    error = (y_true - y_pred) ** 2
    return y_pred, error

def backward_pass(x, y_true, y_pred, weights, bias, learning_rate):
    gradient_w = -2 * (y_true - y_pred) * x
    gradient_b = -2 * (y_true - y_pred)
    
    weights -= learning_rate * gradient_w
    bias -= learning_rate * gradient_b

    return weights, bias

In [ ]:
# Learning rate
learning_rate = 0.1

# Iterate over the input data
for i in range(X_train.shape[0]):
    x = X_train[i]
    y_true = y_train[i]
    weights = w[1:]
    bias = w[0]

    # Forward pass
    y_pred, error = forward_pass(x, weights, bias, y_true)

    if i % 100 == 0:
       print(f"Observation {i + 1}: Predicted Output: {y_pred}, L2 Loss: {error}")

    # Backward pass
    weights, bias = backward_pass(x, y_true, y_pred, weights, bias, learning_rate)

print("Final Weights:", weights)
print("Final Bias:", bias)

In [ ]:
# Initialize error and accuracy counters
total_error = 0
correct_predictions = 0

for i in range(X_test.shape[0]):
    x = X_test[i]
    y_true = y_test[i]

    # Forward pass
    y_pred, error = forward_pass(x, weights, bias, y_true)

    # Accumulate error and update accuracy counter
    total_error += error
    if y_pred == y_true:
        correct_predictions += 1   

# Calculate average error and accuracy
average_error = total_error / X_test.shape[0]
accuracy = correct_predictions / X_test.shape[0]

print("Average L2 Loss on Test Data:", average_error)
print("Accuracy on Test Data:", accuracy)
